# Capstone Project
This Notebook will be used to develop the Capstone Project from the Data Science Specialization of IMB - Coursera

In [87]:
import pandas as pd
import numpy as np

In [88]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


---
# Assignment 2  
Segmenting and Clustering Neighborhoods in Toronto

In [89]:
import requests

In [90]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [91]:
#Make the scrapping from the url defined above (the url to the wikipedia page)
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

In [92]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [93]:
# links=My_table.find_all('a')
# links
# neigh =[]
# for link in links:
#     neigh.append(link.get('title'))
# neigh

In [94]:
# take 'td' that is the part of the HTML that contains the table
lineas=My_table.find_all('td')
# lineas

In [95]:
# lineas[8]

In [96]:
#Here we clean it. if there is Not Assigned, then it wont find anything sraeching for 'a', and the line is treated as a string to remove the <td></td> and the \n.
#If it contains 'a' it means the line has to be treated to take the Neigbour or Borough in the field 'title'
lineas_new = []
for linea in lineas:
    
    if linea.find('a')==None:
        
        linea=str(linea)
        linea=linea[4:]
        linea=linea[:-5]
        linea=linea.replace("\n","")
        
    else:
        
        linea=linea.findChild()
        linea=linea.get('title')
        linea=str(linea)
        if linea[-9:]==", Toronto":
            linea=linea[:-9]

        
    lineas_new.append(linea)
    


In [97]:
# lineas_new

In [98]:
#Base for the table to make the assignments
Tabla = pd.DataFrame(data=np.zeros((int(len(lineas)/3),3)),columns=["PostCode","Borough","Neighborhood"])
Tabla.head()

,PostCode,Borough,Neighborhood
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [99]:
#Table is created

row=0
i=0
for linea in lineas_new:

    Tabla.iloc[row,i]=linea
    
    i=i+1
    if i==3:
        i=0
        row=row+1
Tabla.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park


Drop the Borough Not assigned

In [100]:
Tabla=Tabla[Tabla['Borough']!="Not assigned"].reset_index(drop=True)
Tabla.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [106]:
Tabla.sort_values(by=["PostCode"],inplace=True)
Tabla.reset_index(drop=True,inplace=True)
Tabla.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek (Toronto)


In [102]:
Tabla[Tabla["Neighborhood"]=="Not assigned"]


,PostCode,Borough,Neighborhood
6,M9A,Queen's Park (Toronto),Not assigned


In [103]:
Tabla.loc[6,"Neighborhood"]=Tabla.loc[6,"Borough"]
Tabla[Tabla["Neighborhood"]=="Not assigned"]

,PostCode,Borough,Neighborhood


In [108]:
Tabla_final=pd.DataFrame(Tabla.groupby(['PostCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)))
Tabla_final.reset_index(inplace=True)
Tabla_final.head(10)

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek (Toronto)"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [109]:
Tabla_final.shape

(103, 3)

## Use Geocoder

In [111]:
import geocoder # import geocoder

ModuleNotFoundError: No module named 'geocoder'

In [ ]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

## Geocoder didn't work, lets use de csv

In [115]:
longlat=pd.read_csv('https://cocl.us/Geospatial_data')
longlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [117]:
Tabla_final.merge(longlat, on="PostCode")

KeyError: 'PostCode'

In [125]:
Tabla_locs=pd.merge(Tabla_final,longlat, left_on="PostCode", right_on="Postal Code",left_index=True)
Tabla_locs.drop("Postal Code",axis=1, inplace=True)
Tabla_locs.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek (Toronto)",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
